In [1]:
import chromadb
from chromadb.utils import embedding_functions
from tqdm import tqdm
import pandas as pds
from IPython.display import clear_output
import os

import pandas as pd
import tensorflow as tf
import time
from SonarEmbeddingFunction import SonarEmbeddingFunction


In [2]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from sonar.inference_pipelines.text import TextToEmbeddingModelPipeline


In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import torch
device = torch.device("mps")
device

device(type='mps')

In [5]:
t2vec_model = TextToEmbeddingModelPipeline(encoder="text_sonar_basic_encoder",
                                           tokenizer="text_sonar_basic_encoder", device=device)

In [6]:
embedding_func = SonarEmbeddingFunction()

In [9]:
embedding_func(['test'])

[array([ 0.01450937, -0.00103681, -0.01157396, ..., -0.01648925,
         0.00374215, -0.00468865])]

In [10]:
df = pd.read_csv("hashtag_donaldtrump.csv", lineterminator='\n')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 970919 entries, 0 to 970918
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   created_at            970919 non-null  object 
 1   tweet_id              970919 non-null  float64
 2   tweet                 970919 non-null  object 
 3   likes                 970919 non-null  float64
 4   retweet_count         970919 non-null  float64
 5   source                970043 non-null  object 
 6   user_id               970919 non-null  float64
 7   user_name             970897 non-null  object 
 8   user_screen_name      970919 non-null  object 
 9   user_description      869651 non-null  object 
 10  user_join_date        970919 non-null  object 
 11  user_followers_count  970919 non-null  float64
 12  user_location         675957 non-null  object 
 13  lat                   445719 non-null  float64
 14  long                  445719 non-null  float64
 15  

In [12]:
df.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:01,1.316529e+18,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,Social Mediaset,3.316176e+08,Tgcom24,MediasetTgcom24,Profilo ufficiale di Tgcom24: tutte le notizie...,...,1067661.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.373216530
2,2020-10-15 00:00:02,1.316529e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8.436472e+06,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,1185.0,Portland,45.520247,-122.674195,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060
3,2020-10-15 00:00:02,1.316529e+18,2 hours since last tweet from #Trump! Maybe he...,0.0,0.0,Trumpytweeter,8.283556e+17,Trumpytweeter,trumpytweeter,"If he doesn't tweet for some time, should we b...",...,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.119649591
4,2020-10-15 00:00:08,1.316529e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,4.741380e+07,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,5393.0,Washington DC,38.894992,-77.036558,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121


In [52]:
CHROMA_DATA_PATH = "chroma_data/"
COLLECTION_NAME = "twitter-election2020"

client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

In [108]:
client.delete_collection(COLLECTION_NAME)

In [109]:
collection = client.create_collection(
    name=COLLECTION_NAME,
    embedding_function=embedding_func
)

In [110]:
collection = client.get_collection(COLLECTION_NAME, embedding_function=embedding_func)

In [111]:
select_df = pd.read_csv("select_df.csv")

In [112]:
embeddings_df = pd.read_csv("embeddings.csv")

In [113]:
embeddings_list = embeddings_df.values.tolist()

In [114]:
metadata_cols = select_df.drop(columns=['cleantext']).to_dict(orient='records')

for i in tqdm(range(len(select_df)), desc="Adding documents"):
    collection.add(
        documents=[select_df["cleantext"].iloc[i]],
        embeddings=[embeddings_list[i]],
        metadatas=[metadata_cols[i]],          
        ids=[f"id{i}"]                         
    )

Adding documents:   0%|          | 0/1000 [00:00<?, ?it/s]

Adding documents: 100%|██████████| 1000/1000 [00:02<00:00, 395.65it/s]


In [115]:
collection.count()

1000

In [116]:
collection.peek(10)

{'ids': ['id0', 'id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', 'id9'],
 'embeddings': array([[ 1.14425588e-02, -2.48416691e-05,  7.52648385e-03, ...,
         -1.71165317e-02,  3.68376169e-03,  4.30903817e-03],
        [ 1.24211181e-02, -5.41155925e-03,  2.16312124e-03, ...,
         -1.96006265e-03,  3.36321624e-04,  4.05356195e-03],
        [-3.52008035e-03, -2.23830997e-04,  6.72221696e-03, ...,
          1.45683289e-02, -2.71658017e-03, -5.62054804e-03],
        ...,
        [ 7.61500094e-03,  1.34095168e-02,  1.43038610e-03, ...,
         -9.82502941e-03,  3.22438055e-03, -1.09395180e-02],
        [-7.24022510e-03, -6.12279633e-03,  1.01705194e-02, ...,
          5.54076349e-03, -1.83556913e-04,  3.76053154e-03],
        [ 1.39138929e-03,  8.84038454e-04, -1.72369927e-03, ...,
          6.36337930e-03, -1.29510248e-02, -8.25977046e-03]]),
 'documents': ['is she one of those over kids who can not be reunited with parents who do care thanks to the trump gop government',
  '

In [119]:
query_results = collection.query(
    query_texts=[""],
    n_results=3
)
query_results


{'ids': [['id292', 'id423', 'id851']],
 'embeddings': None,
 'documents': [['trump', 'trump', 'trump']],
 'uris': None,
 'data': None,
 'metadatas': [[{'collected_at': '2020-11-07 11:37:23.119170',
    'created_at': '2020-11-06 00:04:39',
    'likes': 1.0,
    'retweet_count': 0.0,
    'side': 'Trump',
    'source': 'Twitter for iPhone',
    'tweet': '#Trump https://t.co/9oSywIVD6J',
    'tweet_id': 1.324502924094034e+18,
    'user_description': "Screw your problem. I'm talking about me.",
    'user_followers_count': 1213.0,
    'user_id': 127372730.0,
    'user_join_date': '2010-03-29 00:38:58',
    'user_location': 'Santiago de Chile ',
    'user_name': 'Rubens',
    'user_screen_name': 'rubensyyo'},
   {'collected_at': '2020-11-05 12:17:05.525682',
    'created_at': '2020-11-04 03:39:06',
    'likes': 0.0,
    'retweet_count': 0.0,
    'side': 'Trump',
    'source': 'Twitter for iPhone',
    'tweet': 'انا يمكن لو اتحمس ادرس مثل حماسي مع ترامب كان اموري تمام #Trump',
    'tweet_id': 

In [120]:
len(collection.get(ids='id0', include=['embeddings'])['embeddings'][0])

1024